In [ ]:
!pip install ultralytics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from PIL import Image, UnidentifiedImageError
import os
import shutil
from tqdm import tqdm

# Ajusta estas variables según tu proyecto
source_root = '/content/drive/MyDrive/Big_Data/Proyecto_final/tomato'
target_root = '/content/drive/MyDrive/Big_Data/Proyecto_final/Yolo_dataset'
splits = ['train', 'val']
classes = sorted(os.listdir(os.path.join(source_root, 'train')))
class_dict = {name: i for i, name in enumerate(classes)}

# Crear carpetas destino
for split in splits:
    os.makedirs(os.path.join(target_root, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(target_root, 'labels', split), exist_ok=True)

# Procesar imágenes
for split in splits:
    for cls in classes:
        input_folder = os.path.join(source_root, split, cls)
        img_paths = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

        for img_path in tqdm(img_paths, desc=f'{split}/{cls}'):
            img_name = os.path.basename(img_path)
            out_img_path = os.path.join(target_root, 'images', split, img_name)
            out_lbl_path = os.path.join(target_root, 'labels', split, img_name.rsplit('.', 1)[0] + '.txt')

            try:
                with Image.open(img_path) as im:
                    w, h = im.size
            except UnidentifiedImageError:
                print(f"Imagen no válida: {img_path}")
                continue  # Saltar imagen corrupta

            # Copiar imagen válida
            shutil.copy2(img_path, out_img_path)

            # Escribir etiqueta YOLO (caja completa)
            label_idx = class_dict[cls]
            with open(out_lbl_path, 'w') as f:
                f.write(f'{label_idx} 0.5 0.5 1.0 1.0\n')

In [ ]:
import os
import yaml
from ultralytics import YOLO
import shutil
# Dataset convertido
target_root = '/content/drive/MyDrive/Big_Data/Proyecto_final/Yolo_dataset'

# Lista de clases
classes = sorted(os.listdir('/content/drive/MyDrive/Big_Data/Proyecto_final/tomato/train'))

# Crear YAML
yaml_path = os.path.join(target_root, 'data.yaml')
data = {
    'train': os.path.join(target_root, 'images/train'),
    'val': os.path.join(target_root, 'images/val'),
    'nc': len(classes),
    'names': classes
}
with open(yaml_path, 'w') as f:
    yaml.dump(data, f)


model = YOLO('yolov8n.pt')

# Entrenar con solo el 20% del dataset por facilidad computacional
model.train(
    data=yaml_path,
    epochs=5,
    imgsz=416,
    name='tomato_yolo_20%',
    fraction=0.2
)
print(os.listdir('runs/detect'))



# Ruta del modelo entrenado
modelo_entrenado = "runs/detect/tomato_yolo_20%2/weights/best.pt"

# Ruta destino en tu Drive
ruta_destino = "/content/drive/MyDrive/Big_Data/Proyecto_final/Yolo_entrenado/tomato_yolo_best.pt"

# Copiar el modelo a Drive
shutil.copy(modelo_entrenado, ruta_destino)
print(f"Modelo guardado en: {ruta_destino}")

ruta_modelo = "/content/drive/MyDrive/Big_Data/Proyecto_final/Yolo_entrenado/tomato_yolo_best.pt"
ruta_yaml = "/content/drive/MyDrive/Big_Data/Proyecto_final/Yolo_dataset/data.yaml"

# === CARGAR MODELO ===
model = YOLO(ruta_modelo)

# === VALIDAR EL MODELO ===
results = model.val(data=ruta_yaml)

# === MOSTRAR MATRIZ DE CONFUSIÓN ===
# Encuentra automáticamente la carpeta val más reciente
val_dirs = [d for d in os.listdir('runs/detect') if d.startswith('val')]
val_dirs.sort()
ultima_val = val_dirs[-1]
ruta_matriz = f'runs/detect/{ultima_val}/confusion_matrix.png'

display(Image(filename=ruta_matriz))

# === MÉTRICAS GLOBALES ===
print("\n Métricas globales del modelo:")
print(f"Precision: {results.box.mp:.4f}")
print(f"Recall:    {results.box.mr:.4f}")
print(f"mAP@0.5:          {results.box.map50:.4f}")
print(f"mAP@0.5:0.95:     {results.box.map:.4f}")